# Importing stuff and declaring constants

In [244]:
import os
import pandas as pd
import calendar
import numpy as np

TOTAL_MONTHLY_HRS = 150

# Reading Excel

In [245]:
allocation_sheet_df = pd.read_excel(os.path.join(os.getcwd(), './Main.xlsx'), sheet_name = 'Allocation')
employee_sheet_df = pd.read_excel(os.path.join(os.getcwd(), './Main.xlsx'), sheet_name = 'Employees')
project_sheet_df = pd.read_excel(os.path.join(os.getcwd(), './Main.xlsx'), sheet_name = 'Projects')

# Filtering Data

In [246]:
def get_month_number(month_name):
    months = {month.lower(): index for index, month in enumerate(calendar.month_name) if month}
    return months[month_name.lower()]
allocation_sheet_df['Month'] = [get_month_number(i) for i in allocation_sheet_df['Month']]
allocation_sheet_df.dropna(inplace = True)
allocation_sheet_df.drop_duplicates(inplace = True)
allocation_sheet_df

,Year,Month,Name of Employee,Branch,Project Name,Number of Hours,Percentage of Monthly Hours,Percentage Free This Month
0,2023,1,Ramesh R,Developer,Karim Services,50,27.7,72.3
1,2023,2,Ramesh R,Developer,Starry Artworks,70,27.7,72.3
2,2023,3,Rohit,Marketting,Earthern Pans,35,27.7,72.3
3,2023,1,Ramesh R,Developer,Hessington Oil,10,27.7,72.3
4,2023,3,William,IT,Karim Services,75,27.7,72.3
5,2023,4,Ramcharan,Technical,Skyler Parachutes,56,27.7,72.3
6,2023,1,Suresh,Web Designer,Karim Services,100,27.7,72.3
7,2023,3,Ramesh R,Developer,Skyler Parachutes,20,27.7,72.3
8,2023,1,Ramesh R,Developer,Karim Services,80,27.7,72.3
9,2023,1,Sridhar,Developer,Karim Services,80,27.7,72.3


# Creating Employees Dictionary which has everything
This has all the data about that employee. Now this is a dictionary, and you can use this dictionary which has important data to then plot graphs

In [247]:
employees = {}
# template
# employee = {
#     "name": "",
#     "position": "",
#     "projects": {'name': '', 'months' : months},
# }

for i in range(employee_sheet_df.__len__()):
    employee = {'name' : employee_sheet_df.iloc[i].values[0],
                'branch': employee_sheet_df.iloc[i].values[1],
                'projects': {},
                'monthly_hours' : [0 for i in range(1, 13)],
                'monthly_time_free' : [0 for i in range(1, 13)]
                }
    employees[employee_sheet_df.iloc[i].values[0]] = employee


newdf = []
for _, i in enumerate(employees):
    emp_df = allocation_sheet_df[i == allocation_sheet_df['Name of Employee']]
    # adding projects
    for j in range(len(emp_df)):
        employees[i]['projects'][emp_df.iloc[j]['Project Name']] = {}
        employees[i]['projects'][emp_df.iloc[j]['Project Name']] = {k:v for (k, v) in zip([calendar.month_name[i] for i in range(1, 13)], [0 for i in range(1, 13)])}
    # assigning time for each project
    for j in range(len(emp_df)):
        employees[i]['projects'][emp_df.iloc[j]['Project Name']][calendar.month_name[emp_df['Month'].iloc[j]]] += \
            emp_df.iloc[j]['Number of Hours']
    # assigning monthly hours and free time.
    for j in range(1, 13):
        employees[i]['monthly_hours'][j - 1] += emp_df[emp_df['Month'] == j]['Number of Hours'].sum()
        employees[i]['monthly_time_free'][j - 1] = TOTAL_MONTHLY_HRS - employees[i]['monthly_hours'][j - 1]

In [248]:
employees

{'Ramesh R': {'name': 'Ramesh R',
  'branch': 'Developer',
  'projects': {'Karim Services': {'January': 130,
    'February': 0,
    'March': 0,
    'April': 0,
    'May': 0,
    'June': 0,
    'July': 0,
    'August': 0,
    'September': 0,
    'October': 0,
    'November': 0,
    'December': 0},
   'Starry Artworks': {'January': 0,
    'February': 70,
    'March': 0,
    'April': 0,
    'May': 0,
    'June': 0,
    'July': 0,
    'August': 0,
    'September': 0,
    'October': 0,
    'November': 0,
    'December': 0},
   'Hessington Oil': {'January': 10,
    'February': 0,
    'March': 0,
    'April': 0,
    'May': 0,
    'June': 0,
    'July': 0,
    'August': 0,
    'September': 0,
    'October': 0,
    'November': 0,
    'December': 0},
   'Skyler Parachutes': {'January': 0,
    'February': 0,
    'March': 20,
    'April': 0,
    'May': 0,
    'June': 0,
    'July': 0,
    'August': 0,
    'September': 0,
    'October': 0,
    'November': 0,
    'December': 0}},
  'monthly_hours': 

# Creating Employee df

In [265]:
employee_df = pd.DataFrame(employees).transpose()
# employee_df[[True if 'Karim Services' in i.keys() else False for i in employee_df.projects ]]
employee_df

,name,branch,projects,monthly_hours,monthly_time_free
Ramesh R,Ramesh R,Developer,"{'Karim Services': {'January': 130, 'February'...","[140, 70, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[10, 80, 130, 150, 150, 150, 150, 150, 150, 15..."
Suresh,Suresh,Web Designer,"{'Karim Services': {'January': 100, 'February'...","[100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[50, 150, 150, 150, 150, 150, 150, 150, 150, 1..."
Ramcharan,Ramcharan,Technical,"{'Skyler Parachutes': {'January': 0, 'February...","[0, 0, 0, 56, 0, 0, 0, 0, 0, 0, 0, 0]","[150, 150, 150, 94, 150, 150, 150, 150, 150, 1..."
Rohit,Rohit,Marketting,"{'Earthern Pans': {'January': 0, 'February': 0...","[0, 0, 35, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[150, 150, 115, 150, 150, 150, 150, 150, 150, ..."
William,William,IT,"{'Karim Services': {'January': 0, 'February': ...","[0, 0, 75, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[150, 150, 75, 150, 150, 150, 150, 150, 150, 1..."
Sridhar,Sridhar,Developer,"{'Karim Services': {'January': 80, 'February':...","[80, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[70, 150, 150, 150, 150, 150, 150, 150, 150, 1..."


# Graphs